# Testing Dynamics of ECA

## Test description:
# I classify the rules based on the number of 1s in the diagram after 100 steps. I create a list for each rule. Each of them contains the sum of 1s for an initial configuration of length 10 containing from 1 to 9 1s. Finally, I use the k-means algorithm to divide the rules into classes.

-------------

In [70]:
import numpy as np
from src.eca import ECA
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [51]:
configs = [list(l) for l in np.tri(10, dtype=int)][:9]
configs

[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
 [1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
 [1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
 [1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
 [1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
 [1, 1, 1, 1, 1, 1, 1, 1, 0, 0],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 0]]

In [52]:
automata = [ECA.init_config(config) for config in configs] # initializing automata with given configs

In [130]:
data = []
for rule in range(256):
    rule_test = []
    for automaton in automata:
        automaton.evolve(rule, 100)
        rule_test.append(np.sum(np.array(automaton.history[1:])))
    data.append(rule_test)

---------------

## k-means clustering

In [124]:
data = np.array(data)

scaler = StandardScaler()
scaled_data = scaler.fit_transform(data)

num_clusters = 4

kmeans = KMeans(n_clusters=num_clusters, random_state=42)
kmeans.fit(scaled_data)

cluster_labels = kmeans.labels_


In [125]:
for i, label in enumerate(cluster_labels):
    print(f'Rule {i} - Class {label} : {data[i]}')

Rule 0 - Class 2 : [0 0 0 0 0 0 0 0 0]
Rule 1 - Class 0 : [400 400 400 400 400 400 400 500 500]
Rule 2 - Class 2 : [100 100 100 100 100 100 100 100   0]
Rule 3 - Class 0 : [450 450 450 450 450 450 450 450 500]
Rule 4 - Class 2 : [100   0   0   0   0   0   0   0   0]
Rule 5 - Class 0 : [450 400 400 400 400 400 450 500 500]
Rule 6 - Class 2 : [150 150 150 150 150 150 150 150   0]
Rule 7 - Class 0 : [499 450 450 450 450 450 450 500 500]
Rule 8 - Class 2 : [0 1 1 1 1 1 1 1 1]
Rule 9 - Class 0 : [402 402 401 400 400 400 399 399 399]
Rule 10 - Class 2 : [100 200 200 200 200 200 200 200 100]
Rule 11 - Class 0 : [500 500 500 500 500 500 500 500 499]
Rule 12 - Class 2 : [100 100 100 100 100 100 100 100 100]
Rule 13 - Class 0 : [500 403 499 402 498 401 497 496 496]
Rule 14 - Class 2 : [200 200 200 200 200 200 200 200 199]
Rule 15 - Class 0 : [500 500 500 500 500 500 500 500 500]
Rule 16 - Class 2 : [100 100 100 100 100 100 100 100   0]
Rule 17 - Class 0 : [450 450 450 450 450 450 450 450 500]
Ru

In [128]:
labels = list(range(len(set(cluster_labels))))

classes = {f'Class {label}' : list(np.where(cluster_labels == label)[0]) for label in labels}

In [129]:
for cls in classes:
    print(f'{cls}: {list(classes[cls])}')
    print('----------------------------')

Class 0: [1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 26, 27, 28, 29, 30, 31, 33, 35, 37, 39, 41, 43, 45, 47, 49, 50, 51, 53, 54, 55, 57, 58, 59, 60, 61, 62, 63, 65, 67, 69, 70, 71, 73, 75, 77, 78, 79, 81, 82, 83, 85, 86, 87, 89, 90, 91, 92, 93, 94, 95, 97, 99, 101, 102, 103, 105, 106, 107, 109, 110, 111, 113, 114, 115, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 129, 131, 133, 135, 137, 141, 145, 147, 149, 150, 153, 156, 157, 161, 163, 165, 167, 169, 177, 178, 179, 181, 193, 195, 197, 198, 199, 225]
----------------------------
Class 1: [139, 143, 151, 155, 158, 159, 171, 173, 175, 182, 183, 185, 186, 187, 188, 189, 190, 191, 201, 202, 203, 205, 206, 207, 209, 211, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 227, 229, 230, 231, 233, 234, 235, 237, 238, 239, 241, 242, 243, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255]
----------------------------
Class 2: [0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 32, 34, 36, 38, 40, 42, 44, 46, 48, 52, 56, 64, 66

### I obtained 4 classes in which we can see some patterns:
- Class 0: The sums of ones are constant for all initial configurations. They range around 400-600.
- Class 1: The sums of ones are big and constant for all initial configurations. They range around 700-1000.
- Class 2: The sums of ones are small constant for all initial configurations. They range around 0-300.
- Class 3: The sums of ones increase as the number of ones in the initial configuration increases. They increase gradually from e.g. 0-200 to 900-1000.